## Add Table to a Hosted Feature Service from Template


In [1]:
# Import the ArcGIS API for Python
from arcgis.gis import GIS
# Import the ArcGIS API for Python FeatureLayerCollection manager
from arcgis.features import FeatureLayerCollection
# handy package for copying nested data
import copy

# Connect to the current AGO organization. In ArcGIS Online notebooks, "home" will use the current credentials.
gis = GIS("home")

### ItemID arguments

In [ ]:
# Read in the itemID for the feature service we want to add the table to
target_itemid = "4c962e88dacc4cf8b9e5581f45757a7d"

# Read in the itemID for the feature service we want to use as a template
source_itemid = "026e125e75374fcfbcd576eea08c8964"

target_flc = FeatureLayerCollection.fromitem(target_item)
source_flc = FeatureLayerCollection.fromitem(source_item)

### Get the definition from the template

In [ ]:
# Check if the FLC has any tables
if source_flc.tables:
    # Get the first table if so
    source_table = source_flc.tables[0]
else:
    raise Exception("No tables found in the source service.")

print(source_table)

### Convert the table properties to a dictionary

In [ ]:
tbl_def = dict(source_table.properties)
print(tbl_def)

### Step 1: Add the table definition to the feature service

In [ ]:
target_flc.manager.add_to_definition({"tables": [tbl_def]})

### Step 2: Update the data JSON of the feature service so that the table actually appears in the AGOL UI

In [ ]:
# Get data JSONs for both source and target
source_data = source_item.get_data()
target_data = target_item.get_data() or {}

# Ensure tables array exists in target data
if "tables" not in target_data:
    target_data["tables"] = []

# Copy the full table entry from the source data
source_table_entry = copy.deepcopy(source_data["tables"][0])

### Get the index for the new table to be added

In [2]:
if target_data["tables"]:
    existing_ids = [t["id"] for t in target_data["tables"] if "id" in t]
    new_table_id = max(existing_ids) + 1
else:
    new_table_id = 0

source_table_entry["id"] = new_table_id

NameError: name 'target_data' is not defined

### Perform the update in memory

In [ ]:
# Append
target_data["tables"].append(source_table_entry)
print(target_data)

### Push the update to the HFL

In [ ]:
# Push update to the item
target_item.update(data=target_data)